In [4]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [5]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_agriVision.ipynb


In [6]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [7]:
from scipy import spatial
from sklearn.decomposition import PCA

In [8]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [9]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [10]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [11]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [12]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,628420.70,-1.26,1.72,-1.60,0.23,0.12,0.06,-0.03,-0.01
3,-1.26,91264.61,3.87,-0.24,0.25,-0.01,0.01,0.00,0.00
4,1.72,3.87,15241.42,0.05,0.12,-0.01,0.00,-0.00,0.00
5,-1.60,-0.24,0.05,2077.41,0.03,0.02,-0.01,-0.00,-0.00
6,0.23,0.25,0.12,0.03,432.69,0.00,-0.00,0.00,-0.00
7,0.12,-0.01,-0.01,0.02,0.00,105.42,0.00,-0.00,-0.00
8,0.06,0.01,0.00,-0.01,-0.00,0.00,30.83,-0.00,0.00
9,-0.03,0.00,-0.00,-0.00,0.00,-0.00,-0.00,3.80,-0.00
10,-0.01,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.18


In [13]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,0.00002,-0.00004,0.00001,0.00001,0.00001,-0.00002,-0.00002
3,-0.00001,1.00000,0.00010,-0.00002,0.00004,-0.00000,0.00001,0.00000,0.00002
4,0.00002,0.00010,1.00000,0.00001,0.00005,-0.00000,0.00000,-0.00001,0.00001
5,-0.00004,-0.00002,0.00001,1.00000,0.00003,0.00005,-0.00004,-0.00003,-0.00002
6,0.00001,0.00004,0.00005,0.00003,1.00000,0.00002,-0.00002,0.00001,-0.00002
7,0.00001,-0.00000,-0.00000,0.00005,0.00002,1.00000,0.00002,-0.00001,-0.00002
8,0.00001,0.00001,0.00000,-0.00004,-0.00002,0.00002,1.00000,-0.00003,0.00000
9,-0.00002,0.00000,-0.00001,-0.00003,0.00001,-0.00001,-0.00003,1.00000,-0.00004
10,-0.00002,0.00002,0.00001,-0.00002,-0.00002,-0.00002,0.00000,-0.00004,1.00000


In [14]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

6.677147668274077

In [15]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.38943690e+10 9.11234758e+08 2.48885766e+07 4.50166918e+05
 1.87686212e+04 1.04864811e+03 7.86693695e+01 9.01065979e-01
 8.02580364e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,9.998235e-01,-0.018537,-0.003022,-0.000414,-0.000085,-0.000021,-0.000006,-8.062858e-07,-4.489510e-08
1,1.845554e-02,0.999523,-0.024529,-0.003248,-0.000673,-0.000165,-0.000048,-5.900688e-06,-2.540188e-07
2,3.462509e-03,0.024384,0.999415,-0.023209,-0.004726,-0.001153,-0.000337,-4.175031e-05,-1.875938e-06
3,5.489647e-04,0.003766,0.022881,0.998729,-0.043501,-0.010072,-0.002945,-3.637362e-04,-1.695681e-05
4,1.356597e-04,0.000933,0.005599,0.042580,0.996818,-0.064748,-0.017657,-2.161451e-03,-1.006923e-04
5,4.062171e-05,0.000281,0.001682,0.012350,0.062008,0.992261,-0.106220,-1.163309e-02,-5.399244e-04
6,1.632196e-05,0.000113,0.000678,0.004978,0.023969,0.103905,0.992227,-6.388780e-02,-2.898043e-03
7,3.064084e-06,0.000021,0.000125,0.000915,0.004386,0.018018,0.062049,9.968075e-01,-4.669046e-02
8,2.916565e-07,0.000002,0.000012,0.000085,0.000408,0.001673,0.005720,4.640055e-02,9.989051e-01


In [16]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00017647, 0.00047677, 0.00058467, 0.00127111, 0.00318208,
       0.00773932, 0.00777324, 0.00319249, 0.00109495])